In [ ]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
from finvizfinance.screener.overview import Overview
import alpaca_trade_api as tradeapi
import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import arch
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Read your api key environment variable
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("news_api")

In [ ]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=os.environ["news_api"])

In [ ]:
foverview = Overview()

filters_dict = {
    'Performance':'Today +5%',
    'Current Volume':"Over 5M",
    'Country': 'USA'
}
foverview.set_filter(filters_dict=filters_dict)

tickers = foverview.ScreenerView()
#tickers.sort_values(by='Volume', inplace=True, ascending=False)
tickers.set_index('Ticker', inplace=True )
tickers.head(100)

In [ ]:
def get_sentiment(company):
    stock_headlines = ( newsapi.get_everything(
    q=company,
    language='en',
    sort_by='relevancy'))

    sentiments = []

    for articles in stock_headlines['articles']:
        try:
            text = articles['content']
            results = analyzer.polarity_scores(text)
            compound = results['compound']
            pos = results['pos']
            neu = results['neu']
            neg = results['neg'] 

            #put data into sentiments
            sentiments.append({
                'text':text,
                'Compound':compound,
                'Positive':pos,
                'Negative':neg,
                'Neutral':neu})

        except AttributeError:
            pass
    fininal_sentiments = pd.DataFrame(sentiments)
    if not fininal_sentiments.empty:
        return(fininal_sentiments["Positive"].mean())
    else:
        return(0)

In [ ]:
ticker_sentiment = [] 
for ticker in tickers.index: 
    sentiment = get_sentiment(tickers['Company'][ticker])
    print(sentiment)
    if float(sentiment) >= 0.1:
        ticker_sentiment.append(ticker)
        print(f"{ticker} meets our sentiment requirement")
    else:
        print(f"{ticker} does not meet our sentiment requirement")

In [ ]:
### Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2",
    base_url = "https://paper-api.alpaca.markets"
)
account = api.get_account()
print(account.status)

In [ ]:
#Set time frame for reporting stocks
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days = 2) 
start_time = tomorrow.strftime("%b %d %Y") + " 10:00AM"
next_day = today + datetime.timedelta(days = 1) 
end_time = next_day.strftime("%b %d %Y") + " 1:00PM"

date_from = datetime.datetime.strptime(
    start_time, '%b %d %Y %I:%M%p')
date_to = datetime.datetime.strptime(
    end_time, '%b %d %Y %I:%M%p')

In [ ]:
# Set timeframe to '1D'
timeframe = "1D"

#Set timeframe for last 2 years
start_date = pd.Timestamp("2018-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp(next_day, tz="America/New_York").isoformat()

#Get ticker data
df_ticker = api.get_barset(
    ticker_sentiment,
    timeframe,
    start=start_date,
    end=end_date,
).df

df_ticker.dropna()
df_ticker.tail()


In [ ]:
df_closing_prices = pd.DataFrame()
for ticker in ticker_sentiment: 
    df_closing_prices[ticker] = df_ticker[ticker]["close"]

In [ ]:
df_closing_prices.dropna(inplace=True)
df_closing_prices.tail()

In [ ]:
def hpfilter(ticker):
    ts_noise, ts_trend = sm.tsa.filters.hpfilter(df_ticker[ticker][['close']])

    #create different dataframes for each composition
    ticker_decomp = df_ticker[ticker][['close']].copy()
    ticker_decomp['noise'] = ts_noise
    ticker_decomp['trend'] = ts_trend
    return ts_noise.plot(title='Noise')

In [ ]:
for ticker in ticker_sentiment: 
    print(hpfilter((ticker)))

In [ ]:
price_forcasting = pd.DataFrame()
dfs=[]
for ticker in df_closing_prices: 
    model = ARIMA(df_closing_prices[ticker], order=(1,1,1))
    results = model.fit()
    df = pd.DataFrame(results.forecast(steps=15)[0], columns=[ticker])
    dfs.append(df)

final_df = pd.concat(dfs, axis=1)
final_df.head(20)

In [ ]:
df_closing_list = df_closing_prices.iloc[[-1]].values.tolist()
final_df_list = final_df.iloc[[9]].values.tolist()


In [ ]:
base_price = df_closing_list[0]
enter_price = final_df_list[0]
print(enter_price,base_price)

In [ ]:
investment = 2500

stock_capital = round(investment / len(stock_buys),2)
print(stock_capital)

In [ ]:

stock_buys=[]
number_of_shares = {}
num = 0 
for ticker in df_closing_prices: 
    value = round((enter_price[num] - base_price[num])/ base_price[num] * 100.0,2)
    if value > 7:
        stock_buys.append(ticker)
        number_of_shares[ticker] = math.floor(stock_capital/enter_price[num])
        api.submit_order(
            symbol=ticker,
            side='buy',
            type='limit',
            limit_price=enter_price[num],
            qty=number_of_shares[ticker],
            time_in_force='day',
            take_profit=dict(
                limit_price=enter_price[num]*1.50,
            ),
            stop_loss=dict(
                stop_price=enter_price[num]/1.20,
            )
        )
    num += 1


In [ ]:
print(stock_buys)

In [ ]:
print(number_of_shares)

In [ ]:
print(api.get_clock())